### Task 1

In [ ]:
## Install the required libraries
%pip install requests
%pip install geopy

In [ ]:
## Loading libraries
import json
from urllib.request import urlopen
import csv
from geopy.geocoders import Nominatim

In [ ]:
## Loading the json file and viewing the contents of the data
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"

with urlopen(url) as response:
    source = response.read()
    
data = json.loads(source)
print(json.dumps(data, indent=2))


In [ ]:
## Understanding the structure of the data better
#len(data)
#len(data[0])
#len(data[0].get("restaurants"))

In [ ]:
## Extracing the relevant fields
restaurants = []

geolocator = Nominatim(user_agent="google")

countries = {} #to reduce number of times getting geocode, speed up the process

for i in data:
    for j in i.get("restaurants"):
        city = j.get("restaurant")["location"]["city"]
        country_id = j.get("restaurant")["location"]["country_id"]

        if country_id not in countries:
            location = geolocator.geocode(city, language="en")
            country = location.address.split(',')[-1]
            countries[country_id] = country
        else:
            country = countries[country_id]

        entry = [j.get("restaurant")["id"], j.get("restaurant")["name"], country, 
                city, j.get("restaurant")["user_rating"]["votes"], 
                float(j.get("restaurant")["user_rating"]["aggregate_rating"]), 
                j.get("restaurant")["cuisines"]]
        restaurants.append(entry)
        
    

In [ ]:
## Coverting to csv file
csvheader = ["RESTAURANT ID", "RESTAURANT NAME", "COUNTRY", "CITY",
             "USER RATING VOTES", "USER AGGREGATE RATING", "CUISINES"]

with open("restaurants.csv", "w", encoding="UTF-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(csvheader)
    writer.writerows(restaurants)